In [1]:
%%writefile simpleParalle.py

from mpi4py import MPI
import numpy as np

comm=MPI.COMM_WORLD
size=comm.Get_size()
rank=comm.Get_rank()

p=size # processors numbers
n1, n2=4000, 4000
n1Local=n1//p
# each processor's array shape is (n1Local, n2)
arr2dLocal=np.empty((n1Local, n2), dtype='float64')

if rank==0:
    arr2d=np.random.randn(n1,n2)
    arr2dLocal=arr2d[:n1Local, :n2]
    for i in range(1, p):
        start=i*n1Local
        end=(i+1)*n1Local
        comm.Send(arr2d[start:end, :n2], dest=i, tag=11)
        print(f'rank-{rank} have sent chunk-{i}')
else:
    comm.Recv(arr2dLocal, source=0, tag=11)

# all processor will execute this statement
arr2dLocal[arr2dLocal<0]=0
print(f'rank-{rank} have set Negative to zero!')

if rank==0:
    arr2d[:n1Local, :n2]=arr2dLocal
    for i in range(1, p):
        start=i*n1Local
        end=(i+1)*n1Local
        comm.Recv(arr2d[start:end, :n2], source=i, tag=22)
        print(f'rank-{rank} have received chunk-{i}')
else:
    comm.Send(arr2dLocal, dest=0, tag=22)

Overwriting simpleParalle.py


In [2]:
# 默认调用所有的cores
%time !mpiexec.exe python simpleParalle.py
# %time !mpiexec.exe -n python simpleParalle.py

rank-1 have set Negative to zero!
rank-2 have set Negative to zero!
rank-3 have set Negative to zero!
rank-4 have set Negative to zero!
rank-5 have set Negative to zero!
rank-6 have set Negative to zero!
rank-7 have set Negative to zero!
rank-0 have sent chunk-1
rank-0 have sent chunk-2
rank-0 have sent chunk-3
rank-0 have sent chunk-4
rank-0 have sent chunk-5
rank-0 have sent chunk-6
rank-0 have sent chunk-7
rank-0 have set Negative to zero!
rank-0 have received chunk-1
rank-0 have received chunk-2
rank-0 have received chunk-3
rank-0 have received chunk-4
rank-0 have received chunk-5
rank-0 have received chunk-6
rank-0 have received chunk-7
Wall time: 846 ms
